# Обучение модели Catboost

In [1]:
!pip install catboost

     |████████████████████████████████| 65.7 MB 8.4 kB/s  eta 0:00:01
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [2]:
!pip install optuna

     |████████████████████████████████| 282 kB 9.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 15.1 MB/s eta 0:00:01
     |████████████████████████████████| 155 kB 32.6 MB/s eta 0:00:01
     |████████████████████████████████| 79 kB 12.1 MB/s eta 0:00:01
     |████████████████████████████████| 479 kB 31.2 MB/s eta 0:00:01
     |████████████████████████████████| 133 kB 42.1 MB/s eta 0:00:01
     |████████████████████████████████| 49 kB 11.6 MB/s eta 0:00:01
  Created wheel for Mako: filename=Mako-1.1.4-py2.py3-none-any.whl size=75675 sha256=d05976b222a7507a4bd9c5be332d6661e9ce6aa30998d8403cbcaa898902f933
  Stored in directory: /root/.cache/pip/wheels/2a/60/32/02a16820f96c067f6161ef35c21559f8db52c4158d6602b438
  Created wheel for PrettyTable: filename=prettytable-0.7.2-py3-none-any.whl size=13698 sha256=99376d1bf93a66f895db5769b1d731421ad1b81ebbb833dd5cc7834d7215647c
  Stored in directory: /root/.cache/pip/wheels/8c/76/0b/eb9eb3da7e2335e3577e3f96a0ae9f74f206e26457bd1a2

In [4]:
from itertools import chain
import matplotlib.pyplot as plt
from itertools import combinations
import numpy as np
import pandas as pd
import xgboost as xgb
import catboost as ctb
import optuna
from optuna.samplers import RandomSampler
from optuna.integration import OptunaSearchCV
from optuna.distributions import IntUniformDistribution, UniformDistribution
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from PIL import Image
from IPython.display import display
from sklearn import preprocessing
import warnings
import pickle
from itertools import compress
#import xgbfir
warnings.filterwarnings("ignore")

# Делаю предварительную обработку данных

In [4]:


from itertools import chain

import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn import preprocessing

train_file = 'train.csv'
test_file = 'test.csv'




top111 = ['Field12', 'PersonalField52', 'PersonalField80', 'PersonalField44', 'Field9',
          'PropertyField7', 'PropertyField12', 'CoverageField5B', 'PersonalField42', 'PersonalField45',
          'PersonalField81', 'PropertyField8', 'PersonalField79', 'GeographicField45B', 'PropertyField22',
          'PersonalField75', 'PersonalField31', 'PropertyField19', 'PropertyField31', 'GeographicField11A',
          'PersonalField23', 'GeographicField21B', 'PersonalField4A', 'Field10', 'GeographicField16B',
          'GeographicField20A', 'PersonalField25', 'PersonalField4B', 'PropertyField3', 'GeographicField17A',
          'GeographicField59B', 'GeographicField7B', 'GeographicField8A', 'Year', 'GeographicField6B',
          'PersonalField14',
          'GeographicField45A', 'GeographicField14B', 'SalesField12', 'CoverageField11A', 'CoverageField5A', 'Month',
          'PropertyField33', 'PersonalField5', 'CoverageField11B', 'GeographicField11B', 'GeographicField23B',
          'PropertyField39B', 'CoverageField3A', 'GeographicField1B', 'GeographicField17B', 'PropertyField39A',
          'GeographicField41B', 'CoverageField6A', 'SalesField9', 'PersonalField16', 'PersonalField26',
          'PropertyField24A', 'Field8', 'GeographicField28A', 'CoverageField3B', 'SalesField2A', 'GeographicField19B',
          'GeographicField43A', 'PropertyField16B', 'PropertyField16A', 'PropertyField1B', 'CoverageField1B',
          'PropertyField1A', 'GeographicField48B', 'PersonalField11', 'CoverageField1A', 'PersonalField15',
          'GeographicField5B', 'PropertyField34', 'CoverageField8', 'PersonalField82', 'SalesField2B',
          'PropertyField35', 'CoverageField2B', 'SalesField10', 'PropertyField21A', 'SalesField3', 'CoverageField9',
          'SalesField7', 'Weekday', 'PersonalField13', 'PropertyField21B', 'SalesField6', 'SalesField1A',
          'PersonalField9', 'SalesField4', 'PersonalField12', 'PersonalField27', 'PersonalField10B', 'Field7',
          'SalesField1B', 'PersonalField84', 'PersonalField2', 'PersonalField1', 'SalesField5', 'PersonalField10A',
          'PropertyField37', 'PropertyField29', 'GeographicField4B', 'PropertyField2B', 'GeographicField1A',
          'GeographicField61B', 'Field11', 'PersonalField76', 'PropertyField30']

drop_out = ['GeographicField19B', 'PropertyField7', 'GeographicField17A', 'GeographicField28A',
            'GeographicField21B', 'GeographicField7B', 'CoverageField11B', 'GeographicField6B', 'GeographicField45A',
            'PersonalField25', 'Month', 'CoverageField5A', 'GeographicField8A', 'GeographicField1B',
            'CoverageField6A_CoverageField6B', 'PersonalField23', 'Field11', 'PropertyField2B', 'SalesField12',
            'GeographicField41B',
            'PropertyField16A', 'Field10', 'PropertyField3', 'PropertyField16B', 'GeographicField1A',
            'GeographicField20A', 'PersonalField81', 'GeographicField16B', 'GeographicField59B', 'PersonalField79',
            'CoverageField1A_CoverageField3A', 'CoverageField3B_CoverageField4B', 'PropertyField22',
            'GeographicField61B',
            'CoverageField3A_PropertyField21A', 'PropertyField12', 'CoverageField2A_CoverageField3A',
            'CoverageField2B_CoverageField3B', 'PropertyField8', 'PropertyField30', 'GeographicField14B',
            'PersonalField31',
            'PropertyField21A', 'CoverageField3A_CoverageField4A', 'PropertyField31', 'CoverageField11A',
            'PropertyField19', 'GeographicField45B', 'CoverageField1A', 'PersonalField75',
            'GeographicField8A_GeographicField13A', 'CoverageField3B_PropertyField21B',
            'CoverageField1B_CoverageField3B', 'GeographicField6A_GeographicField13A', 'CoverageField5B',
            'PersonalField42', 'PersonalField45', 'PersonalField76', 'GeographicField6A_GeographicField8A',
            'PersonalField80', 'Field9', 'CoverageField3A', 'CoverageField3B',
            'GeographicField8A_GeographicField11A', 'GeographicField11A_GeographicField13A',
            'GeographicField4B',
            'CoverageField2B', 'Field12', 'PropertyField21B', 'CoverageField1B', 'PersonalField44',
            'GeographicField6A_GeographicField11A', 'PersonalField52']

interactions2way = [
    ("CoverageField1B", "PropertyField21B"),
    ("GeographicField6A", "GeographicField8A"),
    ("GeographicField6A", "GeographicField13A"),
    ("GeographicField8A", "GeographicField13A"),
    ("GeographicField11A", "GeographicField13A"),
    ("GeographicField8A", "GeographicField11A"),
    ("GeographicField6A", "GeographicField11A"),
    ("CoverageField6A", "CoverageField6B"),
    ("CoverageField3A", "CoverageField4A"),
    ("CoverageField2B", "CoverageField3B"),
    ("CoverageField1A", "CoverageField3A"),
    ("CoverageField3B", "CoverageField4B"),
    ("CoverageField2A", "CoverageField3A"),
    ("CoverageField1B", "CoverageField3B"),
    ("CoverageField3B", "PropertyField21B"),
    ("CoverageField3A", "PropertyField21A"),
    ("CoverageField1B", "PropertyField16B"),
    ("Weekday", "SalesField7"),
    ("PersonalField9", "CoverageField6B"),
    ("PersonalField12", "CoverageField6A"),
    ("PropertyField16B", "PropertyField21A"),
    ("PersonalField12", "Field8"),
    ("PropertyField32", "PersonalField9"),
    ("Field6", "CoverageField6A"),
    ("PersonalField12", "CoverageField6A"),
    ("CoverageField6A", "PropertyField34"),
    ("PersonalField33", "PropertyField8"),
    ("CoverageField2A", "CoverageField3B")
]

interactions3way = [('PersonalField23', 'PersonalField9', 'PropertyField37'),
                    ('CoverageField3A', 'PersonalField63', 'PropertyField21A'),
                    ('CoverageField3A', 'CoverageField4A', 'PersonalField76'),
                    ('CoverageField3A', 'CoverageField4A', 'GeographicField62A'),
                    ('CoverageField6A', 'PersonalField69', 'PersonalField9'),
                    ('CoverageField6A', 'PersonalField71', 'PersonalField9'),
                    ('GeographicField10B', 'GeographicField13A', 'PersonalField9'),
                    ('GeographicField8A', 'PersonalField71', 'PersonalField9'),
                    ('CoverageField2B', 'PersonalField75', 'PropertyField16B'),
                    ('CoverageField6A', 'PersonalField49', 'PropertyField29'),
                    ('CoverageField4B', 'PersonalField39', 'PropertyField16B'),
                    ('CoverageField11B', 'PersonalField6', 'SalesField2B'),
                    ('CoverageField11B', 'PersonalField36', 'SalesField2B'),
                    ('CoverageField2B', 'PropertyField16B', 'PropertyField8'),
                    ('CoverageField3A', 'GeographicField21A', 'PropertyField21B'),
                    ('GeographicField11A', 'PersonalField48', 'PersonalField9'),
                    ('CoverageField11B', 'PersonalField26', 'SalesField2B'),
                    ('CoverageField1B', 'CoverageField3A', 'PersonalField61'),
                    ('CoverageField1A', 'PropertyField16A', 'PropertyField36'),
                    ('PersonalField9', 'PropertyField10', 'PropertyField32'),
                    ('GeographicField11A', 'GeographicField62A', 'PersonalField12'),
                    ('Field10', 'PersonalField9', 'PropertyField34'),
                    ('CoverageField2B', 'CoverageField3A', 'PersonalField8'),
                    ('Field11', 'PropertyField34', 'SalesField6'),
                    ('PersonalField19', 'PersonalField60', 'PropertyField8')]

interactions4way = [('Field8', 'PersonalField12', 'PersonalField75', 'PropertyField37'),
                    ('CoverageField6A', 'PersonalField12', 'PropertyField37', 'PropertyField8'),
                    ('Field8', 'PersonalField9', 'PropertyField3', 'PropertyField37'),
                    ('CoverageField6A', 'Field8', 'PersonalField84', 'PersonalField9'),
                    ('CoverageField8', 'PersonalField12', 'PersonalField80', 'PropertyField37'),
                    ('CoverageField8', 'Field8', 'PersonalField12', 'PersonalField84'),
                    ('CoverageField5A', 'GeographicField11A', 'PersonalField9', 'PropertyField37'),
                    ('CoverageField1B', 'CoverageField3B', 'CoverageField5A', 'PropertyField22'),
                    ('CoverageField1A', 'CoverageField3A', 'PersonalField82', 'PropertyField19'),
                    ('CoverageField1A', 'CoverageField3A', 'PersonalField11', 'PropertyField19'),
                    ('CoverageField5A', 'Field8', 'PersonalField12', 'PersonalField42'),
                    ('CoverageField6A', 'Field11', 'PersonalField9', 'PropertyField12'),
                    ('CoverageField6A', 'CoverageField8', 'PropertyField35', 'SalesField3'),
                    ('CoverageField3A', 'PersonalField82', 'PropertyField21A', 'Year'),
                    ('CoverageField1B', 'CoverageField3B', 'PersonalField42', 'PropertyField8'),
                    ('CoverageField1B', 'CoverageField3A', 'PersonalField1', 'PropertyField16A'),
                    ('CoverageField1B', 'CoverageField3B', 'PropertyField22', 'PropertyField8'),
                    ('CoverageField6A', 'PersonalField45', 'PersonalField9', 'PropertyField29'),
                    ('CoverageField5A', 'PersonalField1', 'PropertyField35', 'SalesField3'),
                    ('CoverageField1A', 'CoverageField3A', 'Field12', 'PersonalField27'),
                    ('CoverageField5A', 'CoverageField8', 'Field11', 'PropertyField29'),
                    ('CoverageField3B', 'PersonalField25', 'PersonalField45', 'PropertyField21B'),
                    ('CoverageField2B', 'CoverageField3B', 'GeographicField17A', 'PersonalField5'),
                    ('CoverageField1A', 'CoverageField3A', 'PersonalField75', 'Year'),
                    ('Field11', 'PersonalField12', 'PersonalField25', 'PropertyField30')]

interactions2way_list = list(np.unique(list(chain(*interactions2way))))
interactions3way_list = list(np.unique(list(chain(*interactions3way))))
interactions4way_list = list(np.unique(list(chain(*interactions4way))))

interactions_list = interactions2way_list + interactions3way_list + interactions4way_list
tmp_features = list(np.setdiff1d(interactions_list, top111))

tc_features = []


def get_data():
    global tc_features

    train = pd.read_csv(train_file)
    test = pd.read_csv(test_file)

    y_train = train.QuoteConversion_Flag

    train = train.drop(['QuoteNumber', 'QuoteConversion_Flag'], axis=1)
    test = test.drop('QuoteNumber', axis=1)

    ntrain = train.shape[0]

    train_test = pd.concat((train, test), axis=0)

    train_test['Date'] = pd.to_datetime(train_test['Original_Quote_Date'])

    train_test['Year'] = train_test['Date'].dt.year
    train_test['Month'] = train_test['Date'].dt.month
    train_test['Day'] = train_test['Date'].dt.day
    train_test['Weekday'] = train_test['Date'].dt.dayofweek

    train_test['Field10'] = train_test['Field10'].apply(lambda x: x.replace(',', '')).astype(np.int32)
    train_test['PropertyField37'] = train_test['PropertyField37'].apply(lambda x: -1 if x == ' ' else x)
    train_test['GeographicField63'] = train_test['GeographicField63'].apply(lambda x: -1 if x == ' ' else x)

    train_test = train_test.drop(['Date', 'Original_Quote_Date'], axis=1)
    train_test = train_test.fillna(-1)

    categoricals = [x for x in train_test.columns if train_test[x].dtype == 'object']

    for c in categoricals:
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(train_test[c].values))
        train_test[c] = lbl.transform(list(train_test[c].values))

    train = train_test.iloc[:ntrain, :].copy().reset_index(drop=True)
    test = train_test.iloc[ntrain:, :].copy().reset_index(drop=True)

    features = list(train.columns)
    features = np.intersect1d(features, top111 + tmp_features)

    x_train = train[features].copy()
    x_test = test[features].copy()

    x_train['NaNCount'] = x_train.apply(lambda x: np.sum(x == -1), axis=1)
    x_test['NaNCount'] = x_test.apply(lambda x: np.sum(x == -1), axis=1)

    for A, B in interactions2way:
        feat = "_".join([A, B])
        x_train[feat] = x_train[A] - x_train[B]
        x_test[feat] = x_test[A] - x_test[B]

    for A, B, C in interactions3way:
        feat = "_".join([A, B, C])
        tc_features += [feat]
        x_train[feat] = x_train[A] - x_train[B] - x_train[C]
        x_test[feat] = x_test[A] - x_test[B] - x_test[C]

    for A, B, C, D in interactions4way:
        feat = "_".join([A, B, C, D])
        tc_features += [feat]
        x_train[feat] = x_train[A] - x_train[B] - x_train[C] - x_train[D]
        x_test[feat] = x_test[A] - x_test[B] - x_test[C] - x_test[D]

    x_train.drop(tmp_features, axis=1, inplace=True)
    x_test.drop(tmp_features, axis=1, inplace=True)

    x_train.drop(drop_out[-25:], axis=1, inplace=True)
    x_test.drop(drop_out[-25:], axis=1, inplace=True)

    return np.array(x_train), np.array(y_train), np.array(x_test)




In [13]:
x_train, y_train, x_test = get_data()
print (x_train.shape, x_test.shape)

(260753, 164) (173836, 164)


# Поиск гиперпараметров модели

In [17]:
x_tr,x_tst,y_tr,y_tst=train_test_split(x_train,y_train, test_size=0.33)

In [20]:
catboost=CatBoostClassifier(n_estimators=300,
                            loss_function='Logloss',
                            learning_rate=0.05,
                            logging_level='Silent',
                            random_state=0)

In [21]:
# задаю пространство поиска
param_distributions={'max_depth': IntUniformDistribution(5,12),
                     'rsm': UniformDistribution(0.7,0.9),
                     'l2_leaf_reg': UniformDistribution(3.,60.),
                     'random_strength': UniformDistribution(0.,5.),
                     'border_count': IntUniformDistribution(64,254),
                     'bagging_temperature': UniformDistribution(0.,5.)}

In [22]:
optuna_search=OptunaSearchCV(catboost,
                             param_distributions,
                             scoring='roc_auc',
                             random_state=0,
                             n_trials=10,
                             verbose=1,
                             cv=5)

In [23]:
optuna_search.fit(x_tr,y_tr)

[I 2021-01-20 10:05:54,443] A new study created in memory with name: no-name-a9a606cc-38b5-4c5e-939f-1bd3f64ff650
[I 2021-01-20 10:05:54,444] Searching the best hyperparameters using 174704 samples...
[I 2021-01-20 10:07:37,201] Trial 0 finished with value: 0.9652550523151501 and parameters: {'max_depth': 9, 'rsm': 0.7792869205559974, 'l2_leaf_reg': 21.12281168384727, 'random_strength': 1.4550634918956606, 'border_count': 105, 'bagging_temperature': 2.91705836741812}. Best is trial 0 with value: 0.9652550523151501.
[I 2021-01-20 10:13:00,374] Trial 1 finished with value: 0.9652943524200939 and parameters: {'max_depth': 12, 'rsm': 0.8855124699778562, 'l2_leaf_reg': 29.473296634731533, 'random_strength': 2.319971578058211, 'border_count': 113, 'bagging_temperature': 3.983546476266599}. Best is trial 1 with value: 0.9652943524200939.
[I 2021-01-20 10:18:29,756] Trial 2 finished with value: 0.9652776284159883 and parameters: {'max_depth': 12, 'rsm': 0.8694028817378017, 'l2_leaf_reg': 25.95

OptunaSearchCV(cv=5, enable_pruning=False, error_score=nan,
               estimator=<catboost.core.CatBoostClassifier object at 0x7fbdd61c0e50>,
               max_iter=1000, n_jobs=1, n_trials=10,
               param_distributions={'bagging_temperature': UniformDistribution(high=5.0, low=0.0),
                                    'border_count': IntUniformDistribution(high=254, low=64, step=1),
                                    'l2_leaf_reg': UniformDistribution(high=60.0, low=3.0),
                                    'max_depth': IntUniformDistribution(high=12, low=5, step=1),
                                    'random_strength': UniformDistribution(high=5.0, low=0.0),
                                    'rsm': UniformDistribution(high=0.9, low=0.7)},
               random_state=0, refit=True, return_train_score=False,
               scoring='roc_auc', study=None, subsample=1.0, timeout=None,
               verbose=1)

In [24]:
# печатаем наилучшие значения гиперпараметров
print('Наилучшие значения гирепараметров: {}'.format(optuna_search.best_params_))

# печатаем наилучшие значения AUC
print('Наилучшие значения AUC: {}'.format(optuna_search.best_score_))

# смотрим значение AUC на тестовой выборке
print('AUC на тестовой выборке: {}'.format(roc_auc_score(y_tst,optuna_search.predict_proba(x_tst)[:,1])))



Наилучшие значения гирепараметров: {'max_depth': 12, 'rsm': 0.7860379771139723, 'l2_leaf_reg': 16.71302894206979, 'random_strength': 2.025524436348438, 'border_count': 200, 'bagging_temperature': 1.0575784724614494}
Наилучшие значения AUC: 0.9654661328986325
AUC на тестовой выборке: 0.9670435158811965


# Подбираю шаг обучения 'learning_rate'

In [28]:
catboost2=CatBoostClassifier(n_estimators=2400,
                            max_depth=12,
                            rsm=0.786,
                            l2_leaf_reg=16.71,
                            random_strength=2,
                            border_count=200,
                            bagging_temperature=1.058,
                            loss_function='Logloss',
                            learning_rate=0.05,
                            logging_level='Silent',
                            random_state=0)

In [29]:
# задаем новое пространство поиска
param_distributions2={'learning_rate': UniformDistribution(0.01,.1)}

In [30]:
optuna_search=OptunaSearchCV(catboost2,
                             param_distributions2,
                             scoring='roc_auc',
                             random_state=0,
                             n_trials=10,
                             verbose=1,
                             cv=5)

In [31]:
optuna_search.fit(x_tr,y_tr)

[I 2021-01-20 11:04:02,291] A new study created in memory with name: no-name-f4c40901-1f74-488f-8a3b-fc122c335199
[I 2021-01-20 11:04:02,292] Searching the best hyperparameters using 174704 samples...
[I 2021-01-20 11:42:52,271] Trial 0 finished with value: 0.9657993973893996 and parameters: {'learning_rate': 0.05374425907193954}. Best is trial 0 with value: 0.9657993973893996.
[I 2021-01-20 12:21:23,278] Trial 1 finished with value: 0.9655405144263527 and parameters: {'learning_rate': 0.07171410857599203}. Best is trial 0 with value: 0.9657993973893996.
[I 2021-01-20 12:59:19,058] Trial 2 finished with value: 0.9649734864501488 and parameters: {'learning_rate': 0.0987018445595439}. Best is trial 0 with value: 0.9657993973893996.
[I 2021-01-20 13:38:36,999] Trial 3 finished with value: 0.9666044522599421 and parameters: {'learning_rate': 0.027603676914600028}. Best is trial 3 with value: 0.9666044522599421.
[I 2021-01-20 14:17:26,952] Trial 4 finished with value: 0.9654013254558595 and

OptunaSearchCV(cv=5, enable_pruning=False, error_score=nan,
               estimator=<catboost.core.CatBoostClassifier object at 0x7fbdef8c8b10>,
               max_iter=1000, n_jobs=1, n_trials=10,
               param_distributions={'learning_rate': UniformDistribution(high=0.1, low=0.01)},
               random_state=0, refit=True, return_train_score=False,
               scoring='roc_auc', study=None, subsample=1.0, timeout=None,
               verbose=1)

In [32]:
# печатаем наилучшие значения гиперпараметров
print('Наилучшие значения гирепараметров: {}'.format(optuna_search.best_params_))

# печатаем наилучшие значения AUC
print('Наилучшие значения AUC: {}'.format(optuna_search.best_score_))

# смотрим значение AUC на тестовой выборке
print('AUC на тестовой выборке: {}'.format(roc_auc_score(y_tst,optuna_search.predict_proba(x_tst)[:,1])))



Наилучшие значения гирепараметров: {'learning_rate': 0.026243226765095752}
Наилучшие значения AUC: 0.9667698696464535
AUC на тестовой выборке: 0.968257024306459


# Обучаю модель Catboost

In [11]:


train_file = "train.csv"
test_file = "test.csv"
sample_submission = "sample_submission.csv"
submission_filename1 = "catboost_1.csv"
submission_filename2 = "catboost_2.csv"


ctb_params = {
    'random_state': 0,
    'rsm': 0.786,
    'logging_level': 'Silent',
    'l2_leaf_reg': 16.71,
    'learning_rate': 0.0262,
    'loss_function': 'Logloss',
    'random_strength': 2,
    'bagging_temperature': 1.058,
    'max_depth': 12,
    'border_count': 200,
    'eval_metric': 'AUC',
}

top111 = ['Field12', 'PersonalField52', 'PersonalField80', 'PersonalField44', 'Field9',
          'PropertyField7', 'PropertyField12', 'CoverageField5B', 'PersonalField42', 'PersonalField45',
          'PersonalField81', 'PropertyField8', 'PersonalField79', 'GeographicField45B', 'PropertyField22',
          'PersonalField75', 'PersonalField31', 'PropertyField19', 'PropertyField31', 'GeographicField11A',
          'PersonalField23', 'GeographicField21B', 'PersonalField4A', 'Field10', 'GeographicField16B',
          'GeographicField20A', 'PersonalField25', 'PersonalField4B', 'PropertyField3', 'GeographicField17A',
          'GeographicField59B', 'GeographicField7B', 'GeographicField8A', 'Year', 'GeographicField6B',
          'PersonalField14',
          'GeographicField45A', 'GeographicField14B', 'SalesField12', 'CoverageField11A', 'CoverageField5A', 'Month',
          'PropertyField33', 'PersonalField5', 'CoverageField11B', 'GeographicField11B', 'GeographicField23B',
          'PropertyField39B', 'CoverageField3A', 'GeographicField1B', 'GeographicField17B', 'PropertyField39A',
          'GeographicField41B', 'CoverageField6A', 'SalesField9', 'PersonalField16', 'PersonalField26',
          'PropertyField24A', 'Field8', 'GeographicField28A', 'CoverageField3B', 'SalesField2A', 'GeographicField19B',
          'GeographicField43A', 'PropertyField16B', 'PropertyField16A', 'PropertyField1B', 'CoverageField1B',
          'PropertyField1A', 'GeographicField48B', 'PersonalField11', 'CoverageField1A', 'PersonalField15',
          'GeographicField5B', 'PropertyField34', 'CoverageField8', 'PersonalField82', 'SalesField2B',
          'PropertyField35', 'CoverageField2B', 'SalesField10', 'PropertyField21A', 'SalesField3', 'CoverageField9',
          'SalesField7', 'Weekday', 'PersonalField13', 'PropertyField21B', 'SalesField6', 'SalesField1A',
          'PersonalField9', 'SalesField4', 'PersonalField12', 'PersonalField27', 'PersonalField10B', 'Field7',
          'SalesField1B', 'PersonalField84', 'PersonalField2', 'PersonalField1', 'SalesField5', 'PersonalField10A',
          'PropertyField37', 'PropertyField29', 'GeographicField4B', 'PropertyField2B', 'GeographicField1A',
          'GeographicField61B', 'Field11', 'PersonalField76', 'PropertyField30']

drop_out = ['GeographicField19B', 'PropertyField7', 'GeographicField17A', 'GeographicField28A',
            'GeographicField21B', 'GeographicField7B', 'CoverageField11B', 'GeographicField6B', 'GeographicField45A',
            'PersonalField25', 'Month', 'CoverageField5A', 'GeographicField8A', 'GeographicField1B',
            'CoverageField6A_CoverageField6B', 'PersonalField23', 'Field11', 'PropertyField2B', 'SalesField12',
            'GeographicField41B',
            'PropertyField16A', 'Field10', 'PropertyField3', 'PropertyField16B', 'GeographicField1A',
            'GeographicField20A', 'PersonalField81', 'GeographicField16B', 'GeographicField59B', 'PersonalField79',
            'CoverageField1A_CoverageField3A', 'CoverageField3B_CoverageField4B', 'PropertyField22',
            'GeographicField61B',
            'CoverageField3A_PropertyField21A', 'PropertyField12', 'CoverageField2A_CoverageField3A',
            'CoverageField2B_CoverageField3B', 'PropertyField8', 'PropertyField30', 'GeographicField14B',
            'PersonalField31',
            'PropertyField21A', 'CoverageField3A_CoverageField4A', 'PropertyField31', 'CoverageField11A',
            'PropertyField19', 'GeographicField45B', 'CoverageField1A', 'PersonalField75',
            'GeographicField8A_GeographicField13A', 'CoverageField3B_PropertyField21B',
            'CoverageField1B_CoverageField3B', 'GeographicField6A_GeographicField13A', 'CoverageField5B',
            'PersonalField42', 'PersonalField45', 'PersonalField76', 'GeographicField6A_GeographicField8A',
            'PersonalField80', 'Field9', 'CoverageField3A', 'CoverageField3B',
            'GeographicField8A_GeographicField11A', 'GeographicField11A_GeographicField13A',
            'GeographicField4B',
            'CoverageField2B', 'Field12', 'PropertyField21B', 'CoverageField1B', 'PersonalField44',
            'GeographicField6A_GeographicField11A', 'PersonalField52']

interactions2way = [
    ("CoverageField1B", "PropertyField21B"),
    ("GeographicField6A", "GeographicField8A"),
    ("GeographicField6A", "GeographicField13A"),
    ("GeographicField8A", "GeographicField13A"),
    ("GeographicField11A", "GeographicField13A"),
    ("GeographicField8A", "GeographicField11A"),
    ("GeographicField6A", "GeographicField11A"),
    ("CoverageField6A", "CoverageField6B"),
    ("CoverageField3A", "CoverageField4A"),
    ("CoverageField2B", "CoverageField3B"),
    ("CoverageField1A", "CoverageField3A"),
    ("CoverageField3B", "CoverageField4B"),
    ("CoverageField2A", "CoverageField3A"),
    ("CoverageField1B", "CoverageField3B"),
    ("CoverageField3B", "PropertyField21B"),
    ("CoverageField3A", "PropertyField21A"),
    ("CoverageField1B", "PropertyField16B"),
    ("Weekday", "SalesField7"),
    ("PersonalField9", "CoverageField6B"),
    ("PersonalField12", "CoverageField6A"),
    ("PropertyField16B", "PropertyField21A"),
    ("PersonalField12", "Field8"),
    ("PropertyField32", "PersonalField9"),
    ("Field6", "CoverageField6A"),
    ("PersonalField12", "CoverageField6A"),
    ("CoverageField6A", "PropertyField34"),
    ("PersonalField33", "PropertyField8"),
    ("CoverageField2A", "CoverageField3B")
]

interactions3way = [('PersonalField23', 'PersonalField9', 'PropertyField37'),
                    ('CoverageField3A', 'PersonalField63', 'PropertyField21A'),
                    ('CoverageField3A', 'CoverageField4A', 'PersonalField76'),
                    ('CoverageField3A', 'CoverageField4A', 'GeographicField62A'),
                    ('CoverageField6A', 'PersonalField69', 'PersonalField9'),
                    ('CoverageField6A', 'PersonalField71', 'PersonalField9'),
                    ('GeographicField10B', 'GeographicField13A', 'PersonalField9'),
                    ('GeographicField8A', 'PersonalField71', 'PersonalField9'),
                    ('CoverageField2B', 'PersonalField75', 'PropertyField16B'),
                    ('CoverageField6A', 'PersonalField49', 'PropertyField29'),
                    ('CoverageField4B', 'PersonalField39', 'PropertyField16B'),
                    ('CoverageField11B', 'PersonalField6', 'SalesField2B'),
                    ('CoverageField11B', 'PersonalField36', 'SalesField2B'),
                    ('CoverageField2B', 'PropertyField16B', 'PropertyField8'),
                    ('CoverageField3A', 'GeographicField21A', 'PropertyField21B'),
                    ('GeographicField11A', 'PersonalField48', 'PersonalField9'),
                    ('CoverageField11B', 'PersonalField26', 'SalesField2B'),
                    ('CoverageField1B', 'CoverageField3A', 'PersonalField61'),
                    ('CoverageField1A', 'PropertyField16A', 'PropertyField36'),
                    ('PersonalField9', 'PropertyField10', 'PropertyField32'),
                    ('GeographicField11A', 'GeographicField62A', 'PersonalField12'),
                    ('Field10', 'PersonalField9', 'PropertyField34'),
                    ('CoverageField2B', 'CoverageField3A', 'PersonalField8'),
                    ('Field11', 'PropertyField34', 'SalesField6'),
                    ('PersonalField19', 'PersonalField60', 'PropertyField8')]

interactions4way = [('Field8', 'PersonalField12', 'PersonalField75', 'PropertyField37'),
                    ('CoverageField6A', 'PersonalField12', 'PropertyField37', 'PropertyField8'),
                    ('Field8', 'PersonalField9', 'PropertyField3', 'PropertyField37'),
                    ('CoverageField6A', 'Field8', 'PersonalField84', 'PersonalField9'),
                    ('CoverageField8', 'PersonalField12', 'PersonalField80', 'PropertyField37'),
                    ('CoverageField8', 'Field8', 'PersonalField12', 'PersonalField84'),
                    ('CoverageField5A', 'GeographicField11A', 'PersonalField9', 'PropertyField37'),
                    ('CoverageField1B', 'CoverageField3B', 'CoverageField5A', 'PropertyField22'),
                    ('CoverageField1A', 'CoverageField3A', 'PersonalField82', 'PropertyField19'),
                    ('CoverageField1A', 'CoverageField3A', 'PersonalField11', 'PropertyField19'),
                    ('CoverageField5A', 'Field8', 'PersonalField12', 'PersonalField42'),
                    ('CoverageField6A', 'Field11', 'PersonalField9', 'PropertyField12'),
                    ('CoverageField6A', 'CoverageField8', 'PropertyField35', 'SalesField3'),
                    ('CoverageField3A', 'PersonalField82', 'PropertyField21A', 'Year'),
                    ('CoverageField1B', 'CoverageField3B', 'PersonalField42', 'PropertyField8'),
                    ('CoverageField1B', 'CoverageField3A', 'PersonalField1', 'PropertyField16A'),
                    ('CoverageField1B', 'CoverageField3B', 'PropertyField22', 'PropertyField8'),
                    ('CoverageField6A', 'PersonalField45', 'PersonalField9', 'PropertyField29'),
                    ('CoverageField5A', 'PersonalField1', 'PropertyField35', 'SalesField3'),
                    ('CoverageField1A', 'CoverageField3A', 'Field12', 'PersonalField27'),
                    ('CoverageField5A', 'CoverageField8', 'Field11', 'PropertyField29'),
                    ('CoverageField3B', 'PersonalField25', 'PersonalField45', 'PropertyField21B'),
                    ('CoverageField2B', 'CoverageField3B', 'GeographicField17A', 'PersonalField5'),
                    ('CoverageField1A', 'CoverageField3A', 'PersonalField75', 'Year'),
                    ('Field11', 'PersonalField12', 'PersonalField25', 'PropertyField30')]

interactions2way_list = list(np.unique(list(chain(*interactions2way))))
interactions3way_list = list(np.unique(list(chain(*interactions3way))))
interactions4way_list = list(np.unique(list(chain(*interactions4way))))

interactions_list = interactions2way_list + interactions3way_list + interactions4way_list
tmp_features = list(np.setdiff1d(interactions_list, top111))

tc_features = []


def get_data():
    global tc_features

    train = pd.read_csv(train_file)
    test = pd.read_csv(test_file)

    y_train = train.QuoteConversion_Flag

    train = train.drop(['QuoteNumber', 'QuoteConversion_Flag'], axis=1)
    test = test.drop('QuoteNumber', axis=1)

    ntrain = train.shape[0]

    train_test = pd.concat((train, test), axis=0)

    train_test['Date'] = pd.to_datetime(train_test['Original_Quote_Date'])

    train_test['Year'] = train_test['Date'].dt.year
    train_test['Month'] = train_test['Date'].dt.month
    train_test['Day'] = train_test['Date'].dt.day
    train_test['Weekday'] = train_test['Date'].dt.dayofweek

    train_test['Field10'] = train_test['Field10'].apply(lambda x: x.replace(',', '')).astype(np.int32)
    train_test['PropertyField37'] = train_test['PropertyField37'].apply(lambda x: -1 if x == ' ' else x)
    train_test['GeographicField63'] = train_test['GeographicField63'].apply(lambda x: -1 if x == ' ' else x)

    train_test = train_test.drop(['Date', 'Original_Quote_Date'], axis=1)
    train_test = train_test.fillna(-1)

    categoricals = [x for x in train_test.columns if train_test[x].dtype == 'object']

    for c in categoricals:
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(train_test[c].values))
        train_test[c] = lbl.transform(list(train_test[c].values))

    train = train_test.iloc[:ntrain, :].copy().reset_index(drop=True)
    test = train_test.iloc[ntrain:, :].copy().reset_index(drop=True)

    features = list(train.columns)
    features = np.intersect1d(features, top111 + tmp_features)

    x_train = train[features].copy()
    x_test = test[features].copy()

    x_train['NaNCount'] = x_train.apply(lambda x: np.sum(x == -1), axis=1)
    x_test['NaNCount'] = x_test.apply(lambda x: np.sum(x == -1), axis=1)

    for A, B in interactions2way:
        feat = "_".join([A, B])
        x_train[feat] = x_train[A] - x_train[B]
        x_test[feat] = x_test[A] - x_test[B]

    for A, B, C in interactions3way:
        feat = "_".join([A, B, C])
        tc_features += [feat]
        x_train[feat] = x_train[A] - x_train[B] - x_train[C]
        x_test[feat] = x_test[A] - x_test[B] - x_test[C]

    for A, B, C, D in interactions4way:
        feat = "_".join([A, B, C, D])
        tc_features += [feat]
        x_train[feat] = x_train[A] - x_train[B] - x_train[C] - x_train[D]
        x_test[feat] = x_test[A] - x_test[B] - x_test[C] - x_test[D]

    x_train.drop(tmp_features, axis=1, inplace=True)
    x_test.drop(tmp_features, axis=1, inplace=True)

    x_train.drop(drop_out[-25:], axis=1, inplace=True)
    x_test.drop(drop_out[-25:], axis=1, inplace=True)

    return np.array(x_train), np.array(y_train), np.array(x_test)




In [12]:
# проверяю размерность получившихся датасетов
x_train, y_train, x_test = get_data()
print (x_train.shape, x_test.shape)

(260753, 164) (173836, 164)


In [13]:
# подготавливаю датасеты для последовательного обучения
x_train_tc = x_train.copy()
ntcfeat = len(tc_features)

x_train[:, -ntcfeat:] = 0

ntrain = x_train.shape[0]
iterations = 2500

## Обучение первого этапа

In [16]:
x_train

array([[ 2.,  1., 13., ...,  0.,  0.,  0.],
       [ 5.,  9., 13., ...,  0.,  0.,  0.],
       [ 4.,  6., 25., ...,  0.,  0.,  0.],
       ...,
       [ 5.,  8., 13., ...,  0.,  0.,  0.],
       [ 4.,  8., 13., ...,  0.,  0.,  0.],
       [ 8., 16., 13., ...,  0.,  0.,  0.]])

In [ ]:
# сначала обучаюсь на датасете с обнуленными взаимодействующими признаками
pool = Pool(x_train, label=y_train)

ctdt1 = ctb.train(pool=pool, params=ctb_params, iterations=iterations - 100)

pool_test=Pool(x_test)

In [ ]:
# проверяю результат
submission = pd.read_csv(sample_submission)
submission.iloc[:, 1] = ctdt.predict(pool_test, prediction_type='Probability')[:,1]
submission.to_csv(submission_filename, index=False)

In [17]:
!kaggle competitions submit -c homesite-quote-conversion -f catboost_1.csv -m catboost1

Successfully submitted to Homesite Quote Conversion



  0%|          | 0.00/4.57M [00:00<?, ?B/s]
  0%|          | 8.00k/4.57M [00:00<01:01, 78.2kB/s]
 16%|#5        | 728k/4.57M [00:00<00:36, 111kB/s]  
 36%|###6      | 1.66M/4.57M [00:00<00:19, 158kB/s]
 45%|####5     | 2.06M/4.57M [00:01<00:13, 198kB/s]
 65%|######4   | 2.96M/4.57M [00:01<00:05, 281kB/s]
 90%|######### | 4.12M/4.57M [00:01<00:01, 396kB/s]
100%|##########| 4.57M/4.57M [00:03<00:00, 1.52MB/s]


In [18]:
!kaggle competitions submissions homesite-quote-conversion

fileName                         date                 description             status    publicScore  privateScore  
-------------------------------  -------------------  ----------------------  --------  -----------  ------------  
catboost_1.csv                   2021-01-21 15:37:09  catboost1               complete  0.96876      0.96858       
catboost.csv                     2021-01-21 15:19:31  catboost                complete  0.96870      0.96851       
catboost.csv                     2021-01-21 15:03:59  catboost                complete  0.96863      0.96841       
catboost.csv                     2021-01-21 14:43:37  catboost                complete  0.96874      0.96858       
catboost_2.csv                   2021-01-21 14:05:08  catboost_2              complete  0.96874      0.96858       
catboost_1.csv                   2021-01-21 13:42:18  catboost_1              complete  0.96876      0.96858       
model_fs_26.csv                  2021-01-17 10:42:27  model_fs_26       

Результат - 0.96858 на привате. Это где-то 20-е место. Напомню, что оригинальный результат победителя на одиночной модели - 0.96897 - это 4-е место. 

## Второй этап обучения на полном датасете с 300 итерациями

In [20]:
pool_tc = Pool(x_train_tc, label=y_train)
ctb_params['learning_rate'] = 0.01

In [21]:
x_train_tc

array([[ 2.0000e+00,  1.0000e+00,  1.3000e+01, ..., -1.9000e+01,
        -2.0120e+03, -9.8000e-01],
       [ 5.0000e+00,  9.0000e+00,  1.3000e+01, ..., -8.0000e+00,
        -2.0130e+03, -7.5670e-01],
       [ 4.0000e+00,  6.0000e+00,  2.5000e+01, ..., -7.0000e+00,
        -2.0130e+03, -7.5670e-01],
       ...,
       [ 5.0000e+00,  8.0000e+00,  1.3000e+01, ..., -7.0000e+00,
        -2.0130e+03, -4.8114e+00],
       [ 4.0000e+00,  8.0000e+00,  1.3000e+01, ..., -1.0000e+01,
        -2.0190e+03, -7.5670e-01],
       [ 8.0000e+00,  1.6000e+01,  1.3000e+01, ..., -8.0000e+00,
        -2.0130e+03, -9.3300e-01]])

In [ ]:
ctdt2 = ctb.train(pool=pool_tc, params=ctb_params, iterations=300, init_model=ctdt1)

In [ ]:
submission = pd.read_csv(sample_submission)
submission.iloc[:, 1] = ctdt2.predict(pool_test, prediction_type='Probability')[:,1]
submission.to_csv(submission_filename2, index=False)

In [22]:
!kaggle competitions submit -c homesite-quote-conversion -f catboost_2.csv -m catboost2

Successfully submitted to Homesite Quote Conversion



  0%|          | 0.00/4.57M [00:00<?, ?B/s]
  2%|1         | 80.0k/4.57M [00:00<00:06, 740kB/s]
 14%|#4        | 672k/4.57M [00:00<00:04, 1.00MB/s]
 35%|###5      | 1.60M/4.57M [00:00<00:02, 1.37MB/s]
 46%|####5     | 2.09M/4.57M [00:01<00:02, 985kB/s] 
 61%|######1   | 2.80M/4.57M [00:01<00:01, 1.33MB/s]
 84%|########3 | 3.82M/4.57M [00:01<00:00, 1.80MB/s]
100%|##########| 4.57M/4.57M [00:03<00:00, 1.52MB/s]


In [23]:
!kaggle competitions submissions homesite-quote-conversion

fileName                         date                 description             status    publicScore  privateScore  
-------------------------------  -------------------  ----------------------  --------  -----------  ------------  
catboost_2.csv                   2021-01-21 15:52:09  catboost2               complete  0.96874      0.96858       
catboost_1.csv                   2021-01-21 15:37:09  catboost1               complete  0.96876      0.96858       
catboost.csv                     2021-01-21 15:19:31  catboost                complete  0.96870      0.96851       
catboost.csv                     2021-01-21 15:03:59  catboost                complete  0.96863      0.96841       
catboost.csv                     2021-01-21 14:43:37  catboost                complete  0.96874      0.96858       
catboost_2.csv                   2021-01-21 14:05:08  catboost_2              complete  0.96874      0.96858       
catboost_1.csv                   2021-01-21 13:42:18  catboost_1        

После дообучения результат не изменился, причем я пробовал менять learning rate, ставил и больше и меньше, результат только ухудшался

Получается, что модель справилась с задачей и без взаимодействующих признаков.

Попробовал дообучить модель выбрав в параметрах для второй модели 'bootstrap_type': 'Bernoulli' и разное количество итераций: 300, 600, 1000, 2400 результат только ухудшился:

In [ ]:
pool = Pool(x_train, label=y_train)
pool_tc = Pool(x_train_tc, label=y_train)

ctdt = ctb.train(pool=pool, params=ctb_params, iterations=iterations - 100)
ctb_params2 = {'bootstrap_type': 'Bernoulli',
    'random_state': 0,
    'rsm': 0.786,
    'logging_level': 'Silent',
    'l2_leaf_reg': 16.71,
    'learning_rate': 0.01,
    'loss_function': 'Logloss',
    'random_strength': 2,
    'max_depth': 12,
    'border_count': 200,
    'eval_metric': 'AUC',
}


ctdt = ctb.train(pool=pool_tc, params=ctb_params2, iterations=2400, init_model=ctdt)

pool_test=Pool(x_test)

In [ ]:
submission = pd.read_csv(sample_submission)
submission.iloc[:, 1] = ctdt.predict(pool_test, prediction_type='Probability')[:,1]
submission.to_csv('catboost_8.csv', index=False)

In [25]:
!kaggle competitions submissions homesite-quote-conversion

fileName                         date                 description             status    publicScore  privateScore  
-------------------------------  -------------------  ----------------------  --------  -----------  ------------  
catboost_8.csv                   2021-01-21 21:03:57  catboost8               complete  0.96668      0.96621       
catboost_7.csv                   2021-01-21 20:45:49  catboost7               complete  0.96487      0.96407       
catboost_6.csv                   2021-01-21 20:23:57  catboost6               complete  0.95566      0.95408       
catboost_5.csv                   2021-01-21 19:57:31  catboost5               complete  0.96078      0.95951       
catboost_4.csv                   2021-01-21 19:25:05  catboost4               complete  0.96712      0.96713       
catboost_4.csv                   2021-01-21 19:02:39  catboost4               complete  0.96871      0.96855       
catboost_3.csv                   2021-01-21 16:08:24  catboost3         